### 🔎 서포트벡터 분류기
- 리뷰길이 MinMaxScaler 적용

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

df = pd.read_csv('./shampoo_final.csv')
df['긍정부정'] = df['긍정부정'].replace({'y': 1, 'n': 0})

print(df.shape)
df.head()

(10279, 4)


,리뷰,평점,긍정부정,리뷰길이
0,진짜 머리 감고 나면 너무 너무 간지러워 요상쾌함이 오래가지도 못하고 냄새도 빨리 나요,1,0,52
1,비듬 생겼어요 어쩐지간지럽더라 개나 샀는데 누구 주지도 못하고 아오,1,0,50
2,뚜껑 펌프 고장 났네요 아무리 눌러도 안 나와서 뚜껑 열고 사용하다 방치했어요,1,0,40
3,머리카락에 부담이 안 되고 머리 피부에 부담이 안 되니 좋죠,1,0,27
4,샴푸는 사용해보고 살수가 없으니 오랜 기간 동안 고민했고 리뷰들도 꼼꼼히 보았는데 ...,1,0,510


In [2]:
std1 = (df['긍정부정'] == 0) & ((df['평점'] == 4) | (df['평점'] == 5))
std2 = (df['긍정부정'] == 1) & ((df['평점'] == 1) | (df['평점'] == 2) | (df['평점'] == 3))

df.loc[std1, '평점'] = 2
df.loc[std2, '평점'] = 4

df['평점'].value_counts()

평점
4    3507
5    2662
2    1626
3    1324
1    1160
Name: count, dtype: int64

In [3]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()

review_len_scaled = mms.fit_transform(
  np.array(df['리뷰길이']).reshape((-1, 1)))

df['리뷰길이'] = np.array(review_len_scaled).reshape(-1, )

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
  df[['리뷰', '긍정부정', '리뷰길이']], df['평점'],
  stratify=df['평점'],
  test_size=.25)

print("Train Size: ", len(X_train))
print("Test Size: ", len(X_test))

Train Size:  7709
Test Size:  2570


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

t = Okt()

def tokenizer1(txt):
  result = []
  for word, tag in t.pos(txt, norm=True, stem=True):
    if tag in ['Noun', 'Verb', 'Adjective']:
      result.append(word)
  return result

tfidf1 = TfidfVectorizer(
  tokenizer=tokenizer1, max_features=3000,
  min_df=5, max_df=.5)

X_train_tfidf1 = tfidf1.fit_transform(X_train['리뷰'])
X_test_tfidf1 = tfidf1.transform(X_test['리뷰'])

/Users/jason/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
import numpy as np

X_train_combined = np.hstack(
  (X_train_tfidf1.toarray(),
    np.array(X_train['긍정부정']).reshape((-1, 1)),
    np.array(X_train['리뷰길이']).reshape((-1, 1))))

X_test_combined = np.hstack(
  (X_test_tfidf1.toarray(),
    np.array(X_test['긍정부정']).reshape((-1, 1)),
    np.array(X_test['리뷰길이']).reshape((-1, 1))))

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

params = {
  'C': [.1, 1, 10, 100],
  'gamma': [.001, .01, .1, 1]
}

sv_clf = SVC(max_iter=1500, random_state=13, kernel = 'rbf')

gs_cv = GridSearchCV(sv_clf, param_grid=params, n_jobs=-1, refit=True)

In [8]:
gs_cv.fit(X_train_combined, y_train)

/Users/jason/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/jason/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/jason/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/jason/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/jason/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:297: Convergenc

GridSearchCV(estimator=SVC(max_iter=1500, random_state=13), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [0.001, 0.01, 0.1, 1]})

In [9]:
gs_cv.best_params_

{'C': 1, 'gamma': 1}

In [10]:
round(gs_cv.best_score_, 4)

0.6791

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

estimator = gs_cv.best_estimator_
pred = estimator.predict(X_test_combined)
print(round(accuracy_score(y_test, pred), 4))
print(round(recall_score(y_test, pred, average='macro'), 4))
print(round(precision_score(y_test, pred, average='macro'), 4))
print(round(f1_score(y_test, pred, average='macro'), 4))

0.6778
0.6185
0.6422
0.6225


In [12]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, pred)

array([[161, 114,  15,   0,   0],
       [ 68, 256,  82,   0,   0],
       [ 33, 172, 126,   0,   0],
       [  0,   0,   0, 759, 118],
       [  0,   0,   0, 226, 440]])

In [13]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           1       0.61      0.56      0.58       290
           2       0.47      0.63      0.54       406
           3       0.57      0.38      0.45       331
           4       0.77      0.87      0.82       877
           5       0.79      0.66      0.72       666

    accuracy                           0.68      2570
   macro avg       0.64      0.62      0.62      2570
weighted avg       0.68      0.68      0.67      2570



In [14]:
pred

array([4, 2, 4, ..., 5, 3, 5])

In [15]:
df

,리뷰,평점,긍정부정,리뷰길이
0,진짜 머리 감고 나면 너무 너무 간지러워 요상쾌함이 오래가지도 못하고 냄새도 빨리 나요,1,0,0.035689
1,비듬 생겼어요 어쩐지간지럽더라 개나 샀는데 누구 주지도 못하고 아오,1,0,0.034290
2,뚜껑 펌프 고장 났네요 아무리 눌러도 안 나와서 뚜껑 열고 사용하다 방치했어요,1,0,0.027292
3,머리카락에 부담이 안 되고 머리 피부에 부담이 안 되니 좋죠,1,0,0.018195
4,샴푸는 사용해보고 살수가 없으니 오랜 기간 동안 고민했고 리뷰들도 꼼꼼히 보았는데 ...,1,0,0.356193
...,...,...,...,...
10274,단백질 샴푸 첫 구매입니다 대 넘어가니 탈모에만 신경이 쓰이고 그래서 탈모샴푸만 사...,5,1,0.154654
10275,개인적으로 단백질 본드 샴푸세트를 몇 년 동안 사용 중인데 머릿결이 윤기가 돌아 요...,5,1,0.155353
10276,두피가 예민하고 각질이 많아 요티트리 샴푸가 좋다고 해서 구매해봤어요 성분이 좋아서...,5,1,0.087474
10277,두피에 좋은 제품들이 거품이 많이 안 난다는데 이 제품은 거품 진짜 풍성하고 세정력...,5,1,0.122463


In [16]:
X_test_combined

array([[0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.03358992],
       [0.18174286, 0.        , 0.        , ..., 0.        , 0.        ,
        0.04268719],
       [0.15446486, 0.        , 0.        , ..., 0.        , 1.        ,
        0.08117565],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.14275717],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.01329601],
       [0.08283202, 0.        , 0.        , ..., 0.        , 1.        ,
        0.16585024]])

In [17]:
X_test

,리뷰,긍정부정,리뷰길이
6441,샴푸가 대용량에 거품도 잘 나서 만족하면서 사용하고 있어요 재구매 의사 있습니다 추천해요,1,0.033590
285,개월 사용 중 두피 지루성 좋다고 했는데 타 제품하고 좋은 점을 모르겠음 가격만 비...,0,0.042687
9397,가격은 일단 타 사이트 랑 비교했을 때 저렴했구요 배송도 빨랐어요 다만 제품 자체의...,1,0.081176
3556,늘 사용하는 제품이에요,1,0.006998
2290,상품은 받았습니다 하지만 샴푸 모두가 재대로 안 닫혀 있네요 그리고 샴푸가 다 조금...,0,0.111267
...,...,...,...
4436,두피가 간지럽고 상태가 안 좋아샀는데 아직은 잘 모르겠어요 여전히 갑지럽고 두피 케...,0,0.037789
9848,똑 떨어진 걸 모르고 급하게 주문해서 받자마자 바로 사용했어요 늘 사용하던 제품 저...,1,0.042687
5013,신랑 선물해줬어요 요즘 스트레스가 많다 그래서 관리하라고 사줬는데 좋아하더라구 요 ...,1,0.142757
1309,다른 샴푸보다 떡이 너무 잘 지더라구여,0,0.013296


In [18]:
final = X_test.drop(['긍정부정', '리뷰길이'], axis=1)
final

,리뷰
6441,샴푸가 대용량에 거품도 잘 나서 만족하면서 사용하고 있어요 재구매 의사 있습니다 추천해요
285,개월 사용 중 두피 지루성 좋다고 했는데 타 제품하고 좋은 점을 모르겠음 가격만 비...
9397,가격은 일단 타 사이트 랑 비교했을 때 저렴했구요 배송도 빨랐어요 다만 제품 자체의...
3556,늘 사용하는 제품이에요
2290,상품은 받았습니다 하지만 샴푸 모두가 재대로 안 닫혀 있네요 그리고 샴푸가 다 조금...
...,...
4436,두피가 간지럽고 상태가 안 좋아샀는데 아직은 잘 모르겠어요 여전히 갑지럽고 두피 케...
9848,똑 떨어진 걸 모르고 급하게 주문해서 받자마자 바로 사용했어요 늘 사용하던 제품 저...
5013,신랑 선물해줬어요 요즘 스트레스가 많다 그래서 관리하라고 사줬는데 좋아하더라구 요 ...
1309,다른 샴푸보다 떡이 너무 잘 지더라구여


In [19]:
final['기존평점'] = y_test
final['재조정평점'] = pred

In [28]:
fives = final.loc[final['기존평점'] == 5]
fives = fives.sort_values(by='재조정평점')

In [29]:
fives

,리뷰,기존평점,재조정평점
9397,가격은 일단 타 사이트 랑 비교했을 때 저렴했구요 배송도 빨랐어요 다만 제품 자체의...,5,4
9848,똑 떨어진 걸 모르고 급하게 주문해서 받자마자 바로 사용했어요 늘 사용하던 제품 저...,5,4
7944,좋아요 매번 구매요,5,4
9326,처음 사용해봤는데 모발도 부드럽고 거품도 적당히 좋네요 굿굿 계속 사용해봐야겠어요,5,4
8027,가격 배송 품질다 좋아요 다음에도 구매할 게요,5,4
...,...,...,...
8729,요즘 스트레스를 받아서 머리카락 빠지는 양이 늘어서 탈모 초기인 것 같아 지금이나마...,5,5
7654,효과가 너무 좋은 맥주 효모샴푸 맥주 효모는 먹어도 건강에 좋지만 머리탈모에 효과가...,5,5
9957,써본 샴푸 중에 가격 대비 만족도가 가장 좋은 샴푸예요이 거 계속 쓰다가 한동안 이...,5,5
9823,향도 좋고 용량도 넉넉해서 만족합니다,5,5


In [30]:
fives.to_csv('테스트_5점.csv', index=False)